In [ ]:
import urllib.request, urllib.error
from bs4 import BeautifulSoup

url1 = 'http://repositorio.lamolina.edu.pe/handle/20.500.12996/26/recent-submissions'
page1 = urllib.request.urlopen (url1)
#print(page.read())

soup1 = BeautifulSoup(page1)
#print(soup)

import re as re 
def next_page_url(url):
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page)
    
    line = soup.find('a',{'class':'next-page-link'})
    line = line.decode().strip()
    offset = re.findall(r'\?offset=.{2}',line)
    url_diff = offset[0]
    
    url = re.sub('offset.*','',url)
    n_url = url + url_diff
    return n_url

url2 = next_page_url(url1)
print(url2)

url3 = next_page_url(url2)
print(url3)